<a href="https://colab.research.google.com/github/DarioRugg/KickLearning/blob/main/merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
from os.path import join 
import pandas as pd
import numpy as np

In [4]:
data_path = join('.', 'drive', 'MyDrive', 'Project', 'Data')

In [9]:
historic = pd.read_csv(join(data_path, 'complete', 'complete_df.csv'))

In [10]:
collabs = pd.read_csv(join(data_path, 'collabs_df.csv'))

In [14]:
set(collabs.columns).difference(set(historic.columns))

{'collabs_avg_score'}

In [19]:
final = pd.merge(historic, collabs[['id','collabs_avg_score']], on='id').drop(columns='collab_names')

In [41]:
def get_statistics_tiers(obs):
  entry = obs.copy()
  var_name = 'tiers_values'
  var = entry[var_name]
  if type(var) == str:
    var_array = np.array(eval(var))
  else:
    var_array = np.array([var])
  
  if len(var_array):
    entry[f"n_tiers"] = len(var_array)
    entry[f"{var_name}_mean"] = var_array.mean()
    entry[f"{var_name}_std"] = var_array.std()
    entry[f"{var_name}_median"] = np.median(var_array)
    entry[f"{var_name}_max"] = var_array.max()
    entry[f"{var_name}_min"] = var_array.min()

  
  return entry

In [43]:
final = final.apply(get_statistics_tiers, axis=1)

In [46]:
text = pd.read_csv(join(data_path, 'text_variables', 'complete_story_embedding.csv'))

In [88]:
cols = list(set(text.columns).difference(set(final.columns)))

In [89]:
cols += ['id']

In [90]:
final = pd.merge(final, text[cols], on='id')

In [94]:
final.columns

Index([' n_gifs', 'backers_count', 'category', 'collabs_avg_score', 'country',
       'created_at', 'creator_bio', 'creator_id', 'currency', 'deadline',
       'disable_communication', 'fb_linked', 'fx_rate', 'goal', 'has_video',
       'historical_backers_max', 'historical_backers_mean',
       'historical_backers_median', 'historical_backers_min',
       'historical_backers_std', 'historical_delta_launch_max',
       'historical_delta_launch_mean', 'historical_delta_launch_median',
       'historical_delta_launch_min', 'historical_delta_launch_std',
       'historical_pledges_max', 'historical_pledges_mean',
       'historical_pledges_median', 'historical_pledges_min',
       'historical_pledges_std', 'historical_projects',
       'historical_success_rate', 'id', 'image', 'launched_at', 'n_collab',
       'n_images', 'n_tiers', 'n_websites', 'pledged', 'project_url', 'risks',
       'state', 'state_changed_at', 'story', 'sub_category', 'tiers_values',
       'tiers_values_max', 'tier

In [101]:
final['risks'] = final['risks'].apply(lambda x: len(x) if type(x) == str else 0)

In [100]:
final['creator_bio'] = final['creator_bio'].apply(lambda x: len(x) if type(x) == str else 0)

In [102]:
final = final.drop(columns = ['story'])

In [44]:
final.to_csv(join(data_path, 'final_dataset.csv'), index=False)